In [17]:
from copy import deepcopy
from operator import attrgetter, methodcaller
import operator


In [44]:
class State():
    
    def __init__(self, first_value=0, secont_value=0, goal_value=3, max_first_size=5, max_second_size=9):
        
        self.first_vidro = first_value
        self.second_vidro = secont_value
        
        self.goal_value = goal_value
        
        self.max_first_size = max_first_size
        self.max_second_size = max_second_size
        self.id_state = 0
        self.f_score = self.score()
        
    
    def score(self):
        self.id_state+=1
        #return min((self.first_vidro/self.goal_value),(self.second_vidro/self.goal_value))
        return self.goal_value-self.first_vidro-self.second_vidro
    
    def isGoal(self):
        return ((self.goal_value == self.first_vidro) or 
                (self.goal_value == self.second_vidro) or 
                (self.goal_value == self.first_vidro + self.second_vidro))
    
    def fill_first(self):
        self.first_vidro = self.max_first_size
        self.f_score = self.score()
        return self
        
    def fill_second(self):
        self.second_vidro = self.max_second_size
        self.f_score = self.score()
        return self
    
    def from_first_to_second(self):
        if self.first_vidro + self.second_vidro > self.max_second_size:
            self.first_vidro -= self.max_second_size - self.second_vidro
            self.second_vidro = self.max_second_size
        else:
            self.second_vidro += self.first_vidro
            self.first_vidro = 0
        self.f_score = self.score()
        return self
    
    def from_second_to_first(self):
        if self.first_vidro + self.second_vidro > self.max_first_size:
            self.second_vidro -= self.max_first_size - self.first_vidro
            self.first_vidro = self.max_first_size
        else:
            self.first_vidro += self.second_vidro
            self.second_vidro = 0
        self.f_score = self.score()
        return self
            
    def empty_first(self):
        self.first_vidro = 0
        self.f_score = self.score()
        return self
        
    def empty_second(self):
        self.second_vidro = 0
        self.f_score = self.score()
        return self
    
    
    def __cmp__(self,other):
        return cmp(self.f_score,other.f_score)
    
    def __eq__(self, other):
        return self.f_score == other.f_score and self.f_score == other.f_score

    def __lt__(self, other):
        return self.f_score < other.f_score

In [45]:
def state_generator(state, beam_width):
    state_list = []
    
    if state.first_vidro != state.max_first_size:
        temp = deepcopy(state)
        state_list.append(temp.fill_first())
    if state.second_vidro != state.max_second_size:
        temp = deepcopy(state)
        state_list.append(temp.fill_second())
    if state.first_vidro != 0:
        temp = deepcopy(state)
        state_list.append(temp.from_first_to_second())
    if state.second_vidro != 0:
        temp = deepcopy(state)
        state_list.append(temp.from_second_to_first())
    if state.first_vidro != 0:
        temp = deepcopy(state)
        state_list.append(temp.empty_first())
    if state.second_vidro != 0:
        temp = deepcopy(state)
        state_list.append(temp.empty_second())
        
    state_list.sort(key =lambda x: x.f_score, reverse = True)    
    
    return state_list[:beam_width]

In [50]:
%%time
state = State(0, 0)
d = {}
list_of_states = [state]
beam_width = 5
visited = set()
while True:
    curent_state = list_of_states.pop(0)
    visited.add(str(str(curent_state.first_vidro)+str(curent_state.second_vidro)))
    print(curent_state.first_vidro, curent_state.second_vidro, curent_state.f_score)
    
    
    if curent_state.isGoal():
        break
    for child_state in state_generator(curent_state, beam_width):
        
        if str(child_state.first_vidro)+str(child_state.second_vidro) not in visited:
            d[str(child_state.first_vidro)+str(child_state.second_vidro)+str(child_state.id_state)] = str(curent_state.first_vidro)+str(curent_state.second_vidro)+str(curent_state.id_state)
            list_of_states.append(child_state)
            visited.add(str(str(child_state.first_vidro)+str(child_state.second_vidro)))

all_movements = []
value = str(curent_state.first_vidro)+str(curent_state.second_vidro)+str(curent_state.id_state)

all_movements.append(value[:2])
while value != str(state.first_vidro)+str(state.second_vidro)+str(state.id_state):
    value = d[value]
    all_movements.append(value[:2])
all_movements = all_movements[::-1]
print("All movements: ", all_movements)

0 0 3
5 0 -2
0 9 -6
0 5 -2
5 9 -11
5 4 -6
5 5 -7
0 4 -1
1 9 -7
4 0 -1
1 0 2
4 9 -10
0 1 2
5 8 -10
5 1 -3
0 8 -5
0 6 -3
5 3 -5
All movements:  ['00', '09', '54', '04', '40', '49', '58', '08', '53']
CPU times: user 3.06 ms, sys: 1.25 ms, total: 4.31 ms
Wall time: 5.2 ms
